# 이더넷 통신을 측정한 asc 파일 처리
- 내용을 확인한다.
- 어떤 식으로 처리할 지 생각한다. 


In [1]:
# import
from pathlib import Path
import pandas as pd

In [2]:
# asc 파일을 줄 별로 읽어오기
dir_asc = Path('.').absolute()/'asc'
file_asc = dir_asc/'Logging.asc'
with open(file_asc, "r") as file:
    lines = file.readlines()

In [3]:
lines[:25] # Display the first 25 lines for verification

['date Fri Sep 13 03:40:25.108 pm 2024\n',
 'base hex  timestamps absolute\n',
 'internal events logged\n',
 '// version 16.2.0\n',
 '// Measurement UUID: b9330f6a-051b-4f4d-a967-2c68124e7f7b\n',
 'Begin TriggerBlock Fri Sep 13 03:40:25.108 pm 2024\n',
 '   0.000000 Start of measurement\n',
 '   0.000471 ETH 1 STAT Link:Link_up Ports:ECU\n',
 '   0.000867 ETH 1 STAT Link:Link_up LinkSpeed:100Mbit/s Physical:IEEE802.3 Duplex:Full MDI:Crossover Connector:RJ45 Ports:Port5\n',
 '   0.001011 ETH 1 STAT Link:Link_down Connector:RJ45 Ports:Port6\n',
 '   0.040818 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:ECU\n',
 '   0.100574 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:Port5\n',
 '   0.100695 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:Port6\n',
 '   0.14081

In [4]:
# 분석 대상 라인들만 추출하기
lines_oi = []
elements_oi = []
for line in lines:
    elements = line.strip().split()

    interesting = True

    # 첫 element가 타임스탬트가 아니면 분석 대상이 아니다. 
    try:
        ts = float(elements[0])
    except ValueError:
        interesting = False

    # Start, STAT, BUSSTATISTIC가 포함된 라인은 분석 대상이 아니다. 
    for element in elements:
        if 'Start' in elements or 'STAT' in element or 'BUSSTATISTIC' in element:
            interesting = False
            break

    # 분석 대상을 lines_oi에 추가한다.
    if interesting:

        # 3번째 element가 Tx인 경우 FCS를 추가한다.
        if elements[3] == 'Tx':
            elements.insert(5, 'FCS:00000000')

        elements_oi.append(elements)
        lines_oi.append(line.strip())

file_new_asc = file_asc.with_suffix('.new.asc')
with open(file_new_asc, "w") as file:
    for line in lines_oi:
        file.write(line + "\n")


In [5]:
elements_oi[:5] # Display the first 5 elements for verification

[['0.161434',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684900004011664EC6122001C612FFFFC351C351001CC3C8FFFFFF010000000CFF01060E0306010001000000',
  'FCS:233b7b26',
  'Ports:Port5',
  'Sim:0'],
 ['0.161434',
  'ETH',
  '1',
  'Tx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684900004011664EC6122001C612FFFFC351C351001CC3C8FFFFFF010000000CFF01060E0306010001000000',
  'FCS:00000000',
  'Ports:ECU',
  'Sim:0'],
 ['1.161448',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684A00004011664DC6122001C612FFFFC351C351001CC3C7FFFFFF010000000CFF01060F0306010001000000',
  'FCS:eb00c426',
  'Ports:Port5',
  'Sim:0'],
 ['1.161449',
  'ETH',
  '1',
  'Tx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684A00004011664DC6122001C612FFFFC351C351001CC3C7FFFFFF010000000CFF01060F0306010001000000',
  'FCS:00000000',
  'Ports:ECU',
  'Sim:0'],
 ['2.161462',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684B00004011

In [6]:
# list를 DataFrame으로 변환하기
df = pd.DataFrame(elements_oi, columns=['ts', 'type', 'ch', 'tx_rx', 'data', 'fcs', 'port', 'sim'])
df['ts'] = df['ts'].astype(float)
df['ch'] = df['ch'].astype(int)

In [7]:
df.head(5) # Display the first 5 rows for verification

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.161434,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:233b7b26,Ports:Port5,Sim:0
1,0.161434,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
2,1.161448,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:eb00c426,Ports:Port5,Sim:0
3,1.161449,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
4,2.161462,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:c0402bfc,Ports:Port5,Sim:0


## 분석을 바탕으로 asc 파일을 처리하는 함수를 만든다.

In [8]:
def extract_lines_oi_from_asc(file_asc: Path):
    """
    Extract lines of interest from an ASC file.
    
    Parameters:
    file_asc (Path): Path to the input ASC file.
    
    Returns:
    list: Lines of interest extracted from the ASC file.
    """
    with open(file_asc, "r") as file:
        lines = file.readlines()

    lines_oi = []
    elements_oi = []
    for line in lines:
        elements = line.strip().split()

        interesting = True

        # 첫 element가 타임스탬트가 아니면 분석 대상이 아니다. 
        try:
            ts = float(elements[0])
        except ValueError:
            interesting = False

        # Start, STAT, BUSSTATISTIC가 포함된 라인은 분석 대상이 아니다. 
        for element in elements:
            if 'Start' in elements or 'STAT' in element or 'BUSSTATISTIC' in element:
                interesting = False
                break

        # 분석 대상을 lines_oi에 추가한다.
        if interesting:

            # 3번째 element가 Tx인 경우 FCS를 추가한다.
            if elements[3] == 'Tx':
                fcs_found = False
                for element in elements[4:]:
                    if 'FCS' in element:
                        fcs_found = True
                        break

                if not fcs_found:
                    elements.insert(5, 'FCS:00000000')

            elements_oi.append(elements)
            lines_oi.append(line.strip())

    return lines_oi, elements_oi

In [9]:
def lines_oi_to_new_asc_and_xlsx(file_asc: Path):
    """
    Convert an ASC file to an XLSX file.
    
    Parameters:
    asc_file (Path): Path to the input ASC file.
    """
    lines_oi, elements_oi = extract_lines_oi_from_asc(file_asc)

    file_new_asc = file_asc.with_suffix('.new.asc')
    with open(file_new_asc, "w") as file:
        for line in lines_oi:
            file.write(line + "\n")

    # list를 DataFrame으로 변환하기
    df = pd.DataFrame(elements_oi, columns=['ts', 'type', 'ch', 'tx_rx', 'data', 'fcs', 'port', 'sim'])
    df['ts'] = df['ts'].astype(float)
    df['ch'] = df['ch'].astype(int)

    # Save DataFrame to XLSX file
    df.to_excel(file_new_asc.with_suffix('.xlsx'), index=False)

    return df

## Logging.asc 처리

In [10]:
file_asc = Path('.').absolute()/'asc'/'Logging.asc'
df = lines_oi_to_new_asc_and_xlsx(file_asc)

In [11]:
df.head(5) # Display the first 5 rows for verification

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.161434,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:233b7b26,Ports:Port5,Sim:0
1,0.161434,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
2,1.161448,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:eb00c426,Ports:Port5,Sim:0
3,1.161449,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
4,2.161462,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:c0402bfc,Ports:Port5,Sim:0


## obd_ethernet_log.asc 처리리

In [12]:
file_asc = Path('.').absolute()/'asc'/'obd_ethernet_log.asc'
df = lines_oi_to_new_asc_and_xlsx(file_asc)


In [13]:
df.head(5) # Display the first 5 rows for verification

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.257025,ETH,1,Rx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:734d6816,Ports:Port6,Sim:0
1,0.257025,ETH,1,Tx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:00000000,Ports:ECU,Sim:0
2,0.257032,ETH,1,Tx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:734d6816,Ports:Port5,Sim:0
3,0.272157,ETH,1,Rx,9d:333300010002A81374BD389D86DD60054E170067110...,FCS:40a51d09,Ports:Port6,Sim:0
4,0.272157,ETH,1,Tx,9d:333300010002A81374BD389D86DD60054E170067110...,FCS:00000000,Ports:ECU,Sim:0
